In [93]:
from openai import OpenAI
from copy import deepcopy
import pandas as pd
import csv
import os
from pathlib import Path
from tqdm import tqdm

In [16]:
data_folder = '.\data' # data
dataset = pd.DataFrame(columns=['sentence_id','sentence','label','lang','split'])

for language in os.listdir(data_folder):
    for filename in os.listdir(f"{data_folder}{os.sep}{language}"):
        if '.tsv' in filename:
            abs_path = f"{data_folder}{os.sep}{language}{os.sep}{filename}"
            df = pd.read_csv(abs_path, sep='\t', quoting=csv.QUOTE_NONE)
            if 'solved_conflict' in df.columns:
                df.drop(columns=['solved_conflict'], inplace=True)
            df['lang'] = language
            df['split'] = Path(filename).stem
            dataset = pd.concat([dataset, df], axis=0)

In [ ]:
dataset = dataset[dataset['lang'] == 'english']

In [20]:
dataset.head()

,sentence_id,sentence,label,lang,split
0,ab677701-ae20-42b4-89f2-ddf1eb71b8b7,(It’s also true that Alien Nation was “a maste...,SUBJ,english,dev_en
1,56164d11-a7f5-4ac5-8dde-681e6f3436e1,It’s all justified in the name of racial “equi...,SUBJ,english,dev_en
2,676d7dfd-f9d9-42fd-9b00-2165f4576cd7,These issues include punishing Sanctuary Citie...,OBJ,english,dev_en
3,f3c8718e-d553-4730-89d5-5077c96de10a,"Only 20 percent of voters support it, while a ...",OBJ,english,dev_en
4,10cb731a-0d62-4e85-b101-a51b48a20219,"In contrast, in 2017 fewer than half of Republ...",OBJ,english,dev_en


In [51]:
train = dataset[dataset['split'].str.contains('train')].copy()
dev = dataset[(dataset['split'].str.contains('dev')) & ~(dataset['split'].str.contains('dev_test'))].copy()
test = dataset[dataset['split'].str.contains('dev_test')].copy()

print(f"Train: {train.shape}")
print(f"Dev: {dev.shape}")
print(f"Test: {test.shape}")

Train: (830, 5)
Dev: (462, 5)
Test: (484, 5)


In [52]:
print(f"Train: {train['label'].value_counts(normalize=True)}")
print(f"Dev: {dev['label'].value_counts(normalize=True)}")
print(f"Test: {test['label'].value_counts(normalize=True)}")

Train: label
OBJ     0.640964
SUBJ    0.359036
Name: proportion, dtype: float64
Dev: label
SUBJ    0.519481
OBJ     0.480519
Name: proportion, dtype: float64
Test: label
OBJ     0.747934
SUBJ    0.252066
Name: proportion, dtype: float64


In [53]:
train.loc[:, 'label'] = train['label'].apply(lambda x: 0 if x == 'OBJ' else 1)
dev.loc[:, 'label'] = dev['label'].apply(lambda x: 0 if x == 'OBJ' else 1)
test.loc[:, 'label'] = test['label'].apply(lambda x: 0 if x == 'OBJ' else 1)

In [54]:
train['label'] = train['label'].astype(int)
dev['label'] = dev['label'].astype(int)
test['label'] = test['label'].astype(int)

In [55]:
API_KEY = os.environ["DEEPSEEK_API_KEY"]
client = OpenAI(api_key=API_KEY, base_url="https://api.deepseek.com")

In [87]:
base_prompt = [
    {
        'role': 'system',
        'content': 'You are an annotator for subjectivity detection.'
    },
    {
        'role': 'user',
        'content': """Your task is to classify input text as containing subjective or objective claim.
        A sentence is subjective if its content is based on or influenced by personal feelings, tastes, or opinions (asnwer with YES). Otherwise, the sentence is objective (answer with NO).
        More precisely, a sentence is subjective if one or more of the following conditions apply:
        1. expresses an explicit personal opinion from the author (e.g., speculations to draw conclusions);
        2. includes sarcastic or ironic expressions;
        3. gives exhortations of personal auspices;
        4. contains discriminating or downgrading expressions;
        5. contains rhetorical figures that convey the author’s opinion.

        The following ambiguous cases are objective: thirdparty’s opinions, comments that do not draw conclusions and leave open questions, and factual conclusions.
        Note 1: Reported speech verbatim cannot contain elements that we identify as markers of subjectivity as it is not content created by thewriter, and, thus, is objective.
        Note 2: Personal feelings, emotions, or mood of the author, without conveying opinions on the matter, are considered objective since the author is the most reliable source for information regarding their own emotions. Emotion-carrying statements are not excluded since they frequently occur in news articles and excluding them from the corpus would turn it less useful in real application scenarios.

        Respond only YES or NO.

        TEXT:
        {text}

        ANSWER:
        """
    }
]

In [88]:
def prepare_prompts(texts: pd.Series, prompt_template: list,):
    """
    Prepares prompts by applying a chat template to a series of input texts.

    Args:
        texts (pd.Series): A pandas Series containing the input texts.
        prompt_template (list): A list representing the prompt template with placeholders.
        tokenizer (AutoTokenizer): An instance of the AutoTokenizer used to tokenize the prompts.

    Returns:
        list: A list of tokenized prompts ready for generation.
    """
    # Store the prompts
    prompts = []
    # Iterate over the input texts
    for text in texts:

        # Create a deepcopy of the prompt template
        prompt_with_text = deepcopy(prompt_template)

        # Replace the placeholder with the input text
        prompt_with_text[1]['content'] =  prompt_with_text[1]['content'].replace('{text}', text)

        prompts.append(prompt_with_text)

    return prompts

In [89]:
def process_response(response) -> int:
    """
    Processes the response tensor and determines if it contains a positive indication.

    Args:
        response (torch.Tensor): The tensor containing the response from the model.
        tokenizer (AutoTokenizer): The tokenizer used to decode the response tensor.

    Returns:
        int: Returns 1 if the decoded response contains 'YES' after 'ANSWER', otherwise returns 0.
    """

    if 'YES' in response:
        return 1
    elif 'NO' in response:
        return 0
    else:
        return -1 # Model answer is not compliant with the classification task

In [90]:
prompts = prepare_prompts(test['sentence'], base_prompt)

In [ ]:
responses = []
responses_labels = []

for prompt in tqdm(prompts):

    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=prompt,
        stream=False
    )

    responses.append(response.choices[0].message.content)
    responses_labels.append(process_response(response.choices[0].message.content))

  0%|          | 1/484 [00:07<58:39,  7.29s/it]

[0]


  0%|          | 2/484 [00:14<58:50,  7.32s/it]

[0, 0]


  0%|          | 2/484 [00:21<1:25:06, 10.59s/it]


KeyboardInterrupt: 